In [26]:
import time
import odrive
from odrive.enums import *

print("Finding odrv...")
odrv0 = odrive.find_any()

def CALIB():
    print("starting calibration...")
    time.sleep(1)
    odrv0.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
    while odrv0.axis0.current_state != AXIS_STATE_IDLE:
        time.sleep(0.1)
    while odrv0.axis1.current_state != AXIS_STATE_IDLE:
        time.sleep(0.1)

    print("setting params axis 0...")
    odrv0.axis0.encoder.set_linear_count(0)
    odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

    print("setting params axis 1...")
    odrv0.axis1.encoder.set_linear_count(0)
    odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

def STOP():
    odrv0.axis0.requested_state = AXIS_STATE_IDLE
    odrv0.axis1.requested_state = AXIS_STATE_IDLE

def rezero(axis, dir): # 1 = CCW, -1 = CW <-- dir indicates which direction you wish it so spin in while in use
    if axis == 0:
        odrv0.axis0.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
        odrv0.axis0.controller.config.input_mode = INPUT_MODE_PASSTHROUGH
    if axis == 1:
        odrv0.axis1.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
        odrv0.axis1.controller.config.input_mode = INPUT_MODE_PASSTHROUGH

    if axis == 0:
        init = odrv0.axis0.controller.input_pos
        print(init)
        while dir * init > 0:
            odrv0.axis0.controller.input_pos = init
            init += -1 * dir * 0.01
            time.sleep(0.01)
    elif axis == 1:
        init = odrv0.axis1.controller.input_pos
        print(init)
        while dir * init > 0:
            odrv0.axis1.controller.input_pos = init
            init += -1 * dir * 0.01
            time.sleep(0.01)

rezero(axis=0, dir=-1)
rezero(axis=1, dir=1)

import math
L1 = 275
L2 = 275
def q1q2_from_xy(x, y):
    xD_mag = math.sqrt(x*x + y*y)
    c2 = (xD_mag * xD_mag - L1 * L1 - L2 * L2) / (2 * L1 * L2)
    if abs(c2) > 1:
        return None
    if c2 == 1:
        return math.atan2(y, x), 0, math.atan2(y, x), 0
    q2_1 = 1 * math.acos( c2 )
    q2_2 = -1 * math.acos( c2 )
    theta = math.atan2(y, x)
    q1_1 = theta - math.atan2(L2 * math.sin(q2_1), L1 + L2 * math.cos(q2_1))
    q1_2 = theta - math.atan2(L2 * math.sin(q2_2), L1 + L2 * math.cos(q2_2))
    return q1_1, q2_1, q1_2, q2_2
STOP()


Finding odrv...
0.4014337658882141
0.20071688294410706


In [16]:
CALIB()

starting calibration...
setting params axis 0...
setting params axis 1...


In [25]:
import time

time.sleep(1)

odrv0.axis0.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
odrv0.axis0.controller.config.input_mode = INPUT_MODE_PASSTHROUGH
odrv0.axis1.controller.config.control_mode = CONTROL_MODE_POSITION_CONTROL
odrv0.axis1.controller.config.input_mode = INPUT_MODE_PASSTHROUGH

final_standing_height = 350

rezero(0, -1)
rezero(1, 1)

total_time = 0
N = 1
for i in range(N):
    max_q1 = -10000
    min_q1 = 10000
    max_q2 = -10000
    min_q2 = 10000
    t_ = 0
    time_start = time.time()
    time_duration_path_1 = 2
    time_duration_path_2 = 0.01
    final_pos_path_2 = L1 + L2
    time_current = 0

    time_step = 1e-10

    while time_current < time_duration_path_1 + time_duration_path_2:
        t_0 = time.time()
        t_ = time_current 
        q1_1, q2_1, q1_2, q2_2 = 0, 0, 0, 0
        if t_ <= time_duration_path_1: # Down
            x = ((final_standing_height - L1 - L2) / time_duration_path_1) * t_ + (L1 + L2)
            y = 0
            q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
        elif t_ <= time_duration_path_1 + time_duration_path_2: # Up
            slope = ((final_pos_path_2 - final_standing_height) / time_duration_path_2)
            x = slope * t_ + (final_standing_height - slope * time_duration_path_1)
            y = 0
            q1_1, q2_1, q1_2, q2_2 = q1q2_from_xy(x, y)
        odrv0.axis0.controller.input_pos = q2_1 * (10 / (2 * math.pi))
        odrv0.axis1.controller.input_pos = q1_2 * (10 / (2 * math.pi)) # conversion from set-pos -> radians
        max_q1 = max(math.degrees(q1_2), max_q1)
        min_q1 = min(math.degrees(q1_2), min_q1)
        max_q2 = max(math.degrees(q2_2), max_q2)
        min_q2 = min(math.degrees(q2_2), min_q2)
        t_1 = time.time()

        time_current = time.time() - time_start

        t_delta = t_1 - t_0
        if t_delta < time_step:
            time.sleep(time_step - t_delta)

    time_taken = time.time() - time_start
    print(f"Took: {time_taken} on itr {i}")
    total_time += time_taken
    time.sleep(0.1)

print(f"Average time taken per move: {total_time / N}")
print(f"Min q1: {min_q1}, max q1: {max_q1}")
print(f"Min q2: {min_q2}, max q2: {max_q2}")

0.41221609711647034
0.20610804855823517
Took: 2.010535717010498 on itr 0
Average time taken per move: 2.010535717010498
Min q1: 0.0, max q1: 50.46914040313187
Min q2: -100.93828080626373, max q2: 0.0
